In [1]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time
import configparser
import os
import pymysql

In [2]:
config = configparser.ConfigParser()
config.read("/home/yuan/Documents/config.ini")
config.sections()

['office_ssh', 'office_mysql']

In [3]:
modification_site = pd.read_csv("/home/yuan/Documents/pdb_chain_ksites.csv")

In [4]:
modification_site_dic = {}
for i in range(len(modification_site)):
    if modification_site.loc[i,"PDB"] not in modification_site_dic:
        modification_site_dic[modification_site.loc[i,"PDB"]] = {modification_site.loc[i,"CHAIN"]:[modification_site.loc[i,"Ksites"]]}
    elif modification_site.loc[i,"CHAIN"] not in modification_site_dic[modification_site.loc[i,"PDB"]]:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]] = [modification_site.loc[i,"Ksites"]]
    else:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]].append(modification_site.loc[i,"Ksites"])
for protein in modification_site_dic:
    for chain in modification_site_dic[protein]:
        modification_site_dic[protein][chain] = list(modification_site_dic[protein][chain])

In [49]:
unique_list = modification_site[["PDB","CHAIN"]].drop_duplicates()
unique_list = unique_list.reset_index(drop=True)

In [53]:
cd_hit_7 = pd.read_csv("retained_CDHIT_0.7_PTM.csv")

In [54]:
cd_hit_7.columns = ["PDB","CHAIN"]
cd_hit_7 = cd_hit_7["PDB"].drop_duplicates()
cd_hit_7 = cd_hit_7.reset_index(drop=True)
cd_hit_7.head()

0    1ln4
1    1lop
2    1m63
3    1m6v
4    1meo
Name: PDB, dtype: object

In [92]:
cd_hit_7.shape

(539,)

In [84]:
all_elements = line.strip().split("\t")
last_element = all_elements[-1]
Chain = last_element.split(":")[1].split("@")[0]
Component = last_element.split(":")[1].split("@")[1]
Res_type = last_element.split(":")[0][0:3]
Seq = last_element.split(":")[0][3:]
feature = ",".join(all_elements[1:481])

In [85]:
feature

'0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-0.412,1,1,0,0,0,0,0,0,0,0,0,0,33.51,0,0,0,0,1,3,98,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,1,0,2,0,0,0,1,0,0,0,0,-1.7952,6,3,2,1,0,0,0,1,1,0.5,0,0,150.594,1.332,0.333,0.999,1.332,-1,26,837,0,3,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,4,0,0,2,0,3,1,0,0,6,0,0,0,0,0,0,0,0,6,0,0,0,0,2,4,0,0,1,0,0,1,1,0,0,0,0,-0.6059,9,6,1,2,0,0,0,0,1,1.5,0,3,234.339,0.666,0.666,0,0.666,0,15,965,0,1,0,0,0,0,4,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,4,5,0,0,4,0,1,

In [ ]:
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
for oneRow in cd_hit_7[2:]:
    pdbID = oneRow
    if os.path.exists("/mnt/HD1/feature_data/feature_dir/"+pdbID+".ff"):
        print(pdbID)
        with conn as cursor:
            with cursor:
                try:
                    with open("/mnt/HD1/feature_data/feature_dir/"+pdbID+".ff","r") as f:
                        for line in f:
                            if(line[0] == "#"):
                                pass
                            else:
                                all_elements = line.strip().split("\t")
                                last_element = all_elements[-1]
                                Chain = last_element.split(":")[1].split("@")[0]
                                Component = last_element.split(":")[1].split("@")[1]
                                Res_type = last_element.split(":")[0][0:3]
                                Seq = last_element.split(":")[0][3:]
                                feature = ",".join(all_elements[1:481])
                                query = "Insert into FEATURE(feature_ID,PDBID,Chain,Seq,Res_type,Component,\
                                         feature) Values(%s,%s,%s,%s,%s,%s,%s)"
                                value = [all_elements[0],pdbID,Chain,Seq,Res_type,Component,feature]
                                cursor.execute(query,value)
                                
                except Exception as e:
                    print("Exeception occured:{}".format(e))
                    print(pdbID)
    conn.commit()
    
